<a href="https://colab.research.google.com/github/sarina19/Study-Independent/blob/main/JM_4%265_ChatbotAI_Text_to_Speech_ipynb_(Sarina).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Chatbot in Python
> Chatbot sederhana ini menggunakan konsep sederhana penerapan Cosine Similarity

> Disini program Chatbot dibuat dengan sesuai kebutuhan pengguna, jadi kita bisa memilih apakah harus membuat program ini bisa bekerja pada topik tertentu, misalnya untuk Chatbot berbasis Covid-19, atau yang lainnya yang sedang relevan.

In [ ]:
#sumber materi : https://towardsdatascience.com/making-a-simple-and-fast-chatbot-in-10-minutes-2d84a4ba35e
#https://youtu.be/9KZwRBg4-P0

# Tools scraping Artikel

Pada program kali ini kita akan mengambil teks dari internet menggunakan tools newspaper3k, dimana tools ini memiliki beberapa kelebihan yaitu:
1. Kerangka kerja dalam mengunduh artikel yang multi-thread
2. Dapat mengidentifikasi URL baru
3. Mampu mengekstraksi teks dari HTML
4. Terbaik dalam hal ekstraksi gambar dari HTML
5. Mampu mengekstraksi semua gambar dari HTML
6. Dapat mengekstraksi kata kunci dari teks
7. Dapat mengekstrak ringkasan teks
8. Dapat mengekstraksi nama penulis suatu artikel
9. Dapat mengekstraksi istilah yang sedang trend di mesin pencarian Google
10. Dapat bekerja lebih dari 10 bahasa (termasuk Inggris, Jerman, Arab, China, dll)

In [ ]:
!pip install gtts

In [ ]:
!pip install newspaper3k

     |████████████████████████████████| 211 kB 5.3 MB/s 
     |████████████████████████████████| 7.4 MB 43.6 MB/s 
     |████████████████████████████████| 81 kB 8.8 MB/s 
     |████████████████████████████████| 87 kB 6.3 MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13553 sha256=91bdb2a63ed4ab60f28bb62fb6dcd6de0d351baa6f1457533a5f4661ec41560c
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3357 sha256=2f5b031d7a0099bfbfa346862e9feca30aa777e5053aabfb9c28d525ba0143f0
  Stored in directory: /root/.cache/pip/wheels/7f/d4/8f/6e2ca54744c9d7292d88ddb8d42876bcdab5e6d84a21c10346
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398404 sha256=5f5027116c5a91543713cc74a185d503865589b05b8a0194ddae0008d26a948b
  Stored in directory: /root/.cache/pip/wheels/4c/91/46/3c208287b726df325a597957432

In [ ]:
# Import library
from newspaper import Article
import random
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import warnings
import re
warnings.filterwarnings('ignore')

from gtts import gTTS
import IPython.display as ipd
import time
from io import BytesIO

In [ ]:
nltk.download('punkt', quiet=True)

True

# Proses 1 (Membuat korpus dengan mendownload artikel dari sebuah link)

Link dapat diganti dengan link teks berbhasa Indonesia, tetapi kali ini kita mencoba menggunakan teks berbahasa Inggris terlebih dahulu. 

In [ ]:
article = Article('https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521')
article.download()
article.parse()
article.nlp()
corpus = article.text
print(corpus)

Learn more from kidney doctor Andrew Bentall, M.D.

I'm Dr. Andrew Bentall, a kidney doctor at Mayo Clinic. I look after patients with kidney disease, either in the early stages, or with more advanced kidney disease considering dialysis and transplantation as treatment options. In this video, we'll cover the basics of chronic kidney disease. What is it? Who gets it? The symptoms, diagnosis and treatment. Whether you are looking for answers for yourself or for someone you love, we're here to give you the best information available.

Chronic kidney disease is a disease characterized by progressive damage and loss of function in the kidneys. It's estimated that chronic kidney disease affects about one in seven American adults. And most of those don't know they have it. Before we get into the disease itself, let's talk a little bit about the kidneys and what they do. Our kidneys play many important roles keeping our bodies in balance. They remove waste and toxins, excess water from the blo

# Proses 2 (Tokenization)

In [ ]:
# Tokenization
text = corpus
sentence_list = nltk.sent_tokenize(text) #A list of senetences

# Print the list of sentences
print(sentence_list)


['Learn more from kidney doctor Andrew Bentall, M.D.', "I'm Dr. Andrew Bentall, a kidney doctor at Mayo Clinic.", 'I look after patients with kidney disease, either in the early stages, or with more advanced kidney disease considering dialysis and transplantation as treatment options.', "In this video, we'll cover the basics of chronic kidney disease.", 'What is it?', 'Who gets it?', 'The symptoms, diagnosis and treatment.', "Whether you are looking for answers for yourself or for someone you love, we're here to give you the best information available.", 'Chronic kidney disease is a disease characterized by progressive damage and loss of function in the kidneys.', "It's estimated that chronic kidney disease affects about one in seven American adults.", "And most of those don't know they have it.", "Before we get into the disease itself, let's talk a little bit about the kidneys and what they do.", 'Our kidneys play many important roles keeping our bodies in balance.', 'They remove wast

# Proses 3 (Bot Response)

Bot respon digunakan untuk membuat sebuah percakapan awal antara pengguna dengan bot, jadi disini user dapat menyapa bot, nah untuk kata-kata dapat diganti sesuai dengan kebutuhan, misalnya bahasa indonesia. 

Pada percakapan awal tidak menggunakan ML atau metode lainnya, program hanya menjawab dengan kata yang di cocokkan sebagai input.

In [ ]:
#a function to return a random greeting response to a users greeting
def greeting_response(text):
    text = text.lower()
    
    #Bots greeting respone
    bot_greetings = ['howdy','hi','hello','hola']
    
    #Users greeting
    user_greetings = ['hi','hey','hello','greetings','wassup']
    
    for word in text.split():
        if word in user_greetings:
            return random.choice(bot_greetings)
        
    #Random response to greeting
    def gratitude_response(text):
        text=text.lower()
   


1. Pada list kode bot_greetings itu bentuk kata sapaan bot terhadap sapaan yang kita berikan, dan dapat diganti dengan bahasa Indonesia atau lainnya.
2. Sedangkan, pada list user_greetings itu kata sapaan input dari user dan dapat diganti dengan bahasa lainnya.

# Sorting List

In [ ]:
 def index_sort(list_var):
    length = len(list_var)
    list_index = list(range(0, length))
    
    x = list_var        
    for i in range(length):
        for j in range(length):
            if x[list_index[i]] > x[list_index[j]]:
                #swap
                temp = list_index[i]
                list_index[i] = list_index[j]
                list_index[j] = temp
                
    return list_index

# Chatbot Respon Menggunakan Cosine Similarity

Pada respon bot menggunakan teknis Cosine Similarity untuk menghitung kata yang cocok dengan yang di input user terhadap respon dari Bot

In [ ]:
# Creat Bots Response
def bot_response(user_input):
    user_input=user_input.lower()
    sentence_list.append(user_input)
    bot_response= ''
    cm=CountVectorizer().fit_transform(sentence_list)
    similarity_scores=cosine_similarity(cm[-1],cm)
    similarity_scores_list=similarity_scores.flatten()
    index=index_sort(similarity_scores_list)
    index=index[1:]
    response_flag=0
    
    j=0
    for i in range(len(index)):
        if similarity_scores_list[index[i]]>0.0:
            bot_response=bot_response+' '+sentence_list[index[i]]
            response_flag=1
            j=j+1
        if j>2:
            break

        if response_flag==0:
            bot_response=bot_response+" "+"I apologize, I dont understand"

        sentence_list.remove(user_input) 

        return bot_response

Pada listing kode >bot_response=bot_response+" "+"I apologize, I dont understand" pada list kode ini kata I apologize tujuannya untuk memberikan respon karena sistem tidak mengenali kata yang kita input, jadi sebagai bentuk respon bot kepada user. Kata tersebut bisa diganti dengan bahasa Indonesia, misalnya "Maaf saya tidak mengetahui arti atau tidak mengenali kata-kata yang di input".

# Proses 4 (End Process, Making chat box)

In [ ]:
#Start Chat
print("Doc Bot: I am DOc bot and I will answer your queries about chronic kidney disease, if you want to exit type, bye  ?")

exit_list=['exit','bye','see you later','quit']

while(True):
    user_input=input()
    if user_input.lower() in exit_list:
        print('Doc Bot: Bye Bye See you later')
        break
    else:
        if greeting_response(user_input)!= None:
            print('Doc Bot: '+ greeting_response(user_input))
        else:
            print('Doc Bot: '+ bot_response(user_input))
            tts = gTTS(bot_response(user_input), lang='id')
            tts.save('mantap.wav')
            time.sleep(0.08)
            print("Kamu:\n")
            ipd.display(ipd.Audio('mantap.wav', autoplay=True))
            
            print("="*60 + "\n")

Doc Bot: I am DOc bot and I will answer your queries about chronic kidney disease, if you want to exit type, bye  ?
bye
Doc Bot: Bye Bye See you later


Doc Bot ganti dengan ini atau sesuai dengan topik yang lagi di bahas, print("Doc Bot: Mau nanya apa kamu tentang kabel laut?")

In [ ]:
#buat pertanyaan pada kolom diatas seperti : what is chronic kidney disease?, dll.

---
# AI Chatbot Versi Bahasa **Indonesia**
# Proses 1 (Making Corpus with Download an Article)

In [ ]:
article = Article('https://warstek.com/kabel-laut/')
article.download()
article.parse()
article.nlp()
corpus = article.text
print(corpus)

Telkomsel, Indihome, dan Kabel Laut JaSuKa

Pada hari Minggu lalu (19 September 2021), terjadi gangguan jaringan internet pada Telkomsel dan Indihome. Gangguan tersebut menyebabkan penurunan kualitas layanan TelkomGroup baik fixed maupun mobile broadband di beberapa wilayah Indonesia. Di masa pandemi yang menuntut orang untuk berada di rumah seperti ini, gangguan internet tentu menjadi masalah besar. Berbagai kegiatan penting seperti Seleksi Kompetensi Dasar (SKD) CPNS mengalami penundaan. Netizen pun banyak berkomentar negatif terkait hal ini. Saat artikel ini dirilis, gangguan tersebut mungkin belum benar-benar terselesaikan.

Gangguan tersebut disebabkan oleh masalah pada sistem komunikasi kabel laut JaSuKa (Jawa, Sumatera dan Kalimantan) ruas Batam-Pontianak. Telkom memperkirakan proses perbaikan kabel laut yang terganggu akan berlangsung sekitar sebulan. Meski begitu, Telkom mencoba berbagai alternatif agar jaringan internet tetap dapat berjalan sebaik mungkin, meski tentu tidak s

# Proses 2 (Tokenization)

In [ ]:
# Tokenization
text = corpus
sentence_list = nltk.sent_tokenize(text) #A list of senetences

# Print the list of sentences
print(sentence_list)

['Telkomsel, Indihome, dan Kabel Laut JaSuKa\n\nPada hari Minggu lalu (19 September 2021), terjadi gangguan jaringan internet pada Telkomsel dan Indihome.', 'Gangguan tersebut menyebabkan penurunan kualitas layanan TelkomGroup baik fixed maupun mobile broadband di beberapa wilayah Indonesia.', 'Di masa pandemi yang menuntut orang untuk berada di rumah seperti ini, gangguan internet tentu menjadi masalah besar.', 'Berbagai kegiatan penting seperti Seleksi Kompetensi Dasar (SKD) CPNS mengalami penundaan.', 'Netizen pun banyak berkomentar negatif terkait hal ini.', 'Saat artikel ini dirilis, gangguan tersebut mungkin belum benar-benar terselesaikan.', 'Gangguan tersebut disebabkan oleh masalah pada sistem komunikasi kabel laut JaSuKa (Jawa, Sumatera dan Kalimantan) ruas Batam-Pontianak.', 'Telkom memperkirakan proses perbaikan kabel laut yang terganggu akan berlangsung sekitar sebulan.', 'Meski begitu, Telkom mencoba berbagai alternatif agar jaringan internet tetap dapat berjalan sebaik m

In [ ]:
#a function to return a random greeting response to a users greeting
def greeting_response(text):
    text = text.lower()
    
    #Bots greeting respone
    bot_greetings = ['halo','hai','yuhuu','*eyebrows up*']
    
    #Users greeting
    user_greetings = ['Haloo','Eh iyaa Haii','Hai','greetings','wassup']
    
    for word in text.split():
        if word in user_greetings:
            return random.choice(bot_greetings)
        
    #Random response to greeting
    def gratitude_response(text):
        text=text.lower()
   


In [ ]:
 def index_sort(list_var):
    length = len(list_var)
    list_index = list(range(0, length))
    
    x = list_var        
    for i in range(length):
        for j in range(length):
            if x[list_index[i]] > x[list_index[j]]:
                #swap
                temp = list_index[i]
                list_index[i] = list_index[j]
                list_index[j] = temp
                
    return list_index

In [ ]:
# Creat Bots Response
def bot_response(user_input):
    user_input=user_input.lower()
    sentence_list.append(user_input)
    bot_response= ''
    cm=CountVectorizer().fit_transform(sentence_list)
    similarity_scores=cosine_similarity(cm[-1],cm)
    similarity_scores_list=similarity_scores.flatten()
    index=index_sort(similarity_scores_list)
    index=index[1:]
    response_flag=0
    
    j=0
    for i in range(len(index)):
        if similarity_scores_list[index[i]]>0.0:
            bot_response=bot_response+' '+sentence_list[index[i]]
            response_flag=1
            j=j+1
        if j>2:
            break

        if response_flag==0:
            bot_response=bot_response+" "+"I apologize, I dont understand"

        sentence_list.remove(user_input) 

        return bot_response

# Proses 4 (End Process, Making chat box)

In [ ]:
#Start Chat
print("Doc Bot: Mau nanya apa kamu tentang kabel laut")

exit_list=['exit','bye','keluar','quit', 'sampai jumpa']

while(True):
    user_input=input()
    if user_input.lower() in exit_list:
        print('Doc Bot: Bye Bye Sampai jumpa lagi')
        break
    else:
        if greeting_response(user_input)!= None:
            print('Doc Bot: '+ greeting_response(user_input))
            
        else:
            print('Doc Bot: '+ bot_response(user_input))
            tts = gTTS(bot_response(user_input), lang='id')
            tts.save('mantap.wav')
            time.sleep(0.08)
            print("Kamu:\n")
            ipd.display(ipd.Audio('mantap.wav', autoplay=True))
            
            print("="*60 + "\n")

Doc Bot: Mau nanya apa kamu tentang kabel laut
Doc Bot:  Sebagai mahasiswa Teknik Elektro yang pernah sedikit mendalami internet, yang bisa penulis katakan terkait internet adalah “It’s never been simple”

Mengenal Kabel Laut

Ketika mendengar bahwa 99% internet itu melewati kabel laut, apa bayangan kita terkait bentuk fisik dari kabel laut itu?
Kamu:




Doc Bot:  Jaringan Kabel Laut di Indonesia

Kabel laut yang ada di negeri kita juga cukup banyak.
Kamu:



In [ ]:
#Coba berikan input kalimat berikut

# kabel laut adalah
# kabel laut untuk komunikasi internasional adalah